# DuckDB Library Introduction

---

## Import the library and make a first example

In the next code space I just import the DuckDB library and in the next code space you can find the first example on how to use DuckDB.

In [9]:
# Importing the useful libraries
import duckdb

# Install and load the spatial extension
duckdb.query("INSTALL spatial; LOAD spatial;")

In [2]:
# Connect to an in-memory DuckDB database
conn = duckdb.connect(database=':memory:')

# Create a simple table
conn.execute("""
    CREATE TABLE people (
        name VARCHAR,
        age INTEGER
    )
""")

# Insert some data
conn.execute("""
    INSERT INTO people (name, age) VALUES
    ('Alice', 30),
    ('Bob', 25),
    ('Charlie', 35)
""")

# Query the table
result = conn.execute("SELECT * FROM people").fetchall()
print(result)

[('Alice', 30), ('Bob', 25), ('Charlie', 35)]


In [5]:
# Create a persistent database file
conn = duckdb.connect(database='data/example/my_database.duckdb')

# Create a table
conn.execute("CREATE TABLE products (id INTEGER, name VARCHAR)")

# Insert data
conn.execute("INSERT INTO products VALUES (1, 'Laptop'), (2, 'Phone')")

# Query and print
print(conn.execute("SELECT * FROM products").fetchall())

# Close the connection
conn.close()

[(1, 'Laptop'), (2, 'Phone')]


In [ ]:
# Connect to an in-memory database
conn = duckdb.connect(database=':memory:')

# Read a CSV into a DuckDB table
conn.execute("""
    CREATE TABLE my_csv AS
    SELECT * FROM read_csv_auto('path_to_your_file.csv')
""") # THE FILE DOESN'T EXIST...

# Query the table
print(conn.execute("SELECT * FROM my_csv LIMIT 5").fetchall())

In [6]:
conn = duckdb.connect(database=':memory:')

# Create table with some numbers
conn.execute("""
    CREATE TABLE numbers AS
    SELECT range AS num
    FROM range(1, 10)
""")

# Query: filter even numbers
even_numbers = conn.execute("SELECT num FROM numbers WHERE num % 2 = 0").fetchall()
print("Even numbers:", even_numbers)

Even numbers: [(2,), (4,), (6,), (8,)]


In [ ]:
conn = duckdb.connect(database=':memory:')

# Read parquet file directly
parquet_data = conn.execute("""
    SELECT * FROM read_parquet('path_to_your_file.parquet')
""").fetchall() # THE FILE DOESN'T EXIST...

print(parquet_data)

---

# Let's use DuckDB properly for the Spatial Extension

There you can see an introduction to learn how to use DuckDB with Spatial.

In [16]:
# REQUIRED LIBRARIES
import duckdb
import geopandas as gpd
import pandas as pd
import folium
from shapely.geometry import Point
from shapely import wkt
import os

# Read GeoJSON
cities = gpd.read_file("data/cities.geojson")
cities['geometry_wkt'] = cities['geometry'].apply(lambda geom: geom.wkt)

# Read sensors CSV
df = pd.read_csv("data/sensors.csv")
df['geometry'] = df.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
gdf_sensors = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")
gdf_sensors['geometry_wkt'] = gdf_sensors['geometry'].apply(lambda geom: geom.wkt)

# Create connection
con = duckdb.connect()

# INSTALL + LOAD inside this connection
con.execute("INSTALL spatial;")
con.execute("LOAD spatial;")

# Register DataFrames with WKT geometries
con.register("cities", cities[['name', 'geometry_wkt']])
con.register("sensors", gdf_sensors[['id', 'value', 'geometry_wkt']])

# Run query using ST_GeomFromText
result = con.execute("""
    SELECT s.id, s.value, c.name AS city
    FROM sensors s, cities c
    WHERE ST_Within(ST_GeomFromText(s.geometry_wkt), ST_GeomFromText(c.geometry_wkt))
""").fetchdf()

print(result)

   id  value    city
0   1   34.2  Milano
1   3   40.5  Milano


## Starting point: examples and mapping usage of DuckDB + Spatial

In [17]:
# Load GeoJSON of cities
cities = gpd.read_file("data/cities.geojson")
cities['geometry_wkt'] = cities['geometry'].apply(lambda g: g.wkt)

# Load sensors with lat/lon CSV and convert to GeoDataFrame
df = pd.read_csv("data/sensors.csv")  # must contain 'id', 'value', 'lat', 'lon'
df['geometry'] = df.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
gdf_sensors = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")
gdf_sensors['geometry_wkt'] = gdf_sensors['geometry'].apply(lambda g: g.wkt)

con = duckdb.connect()
con.execute("INSTALL spatial;")
con.execute("LOAD spatial;")

con.register("cities", cities[['name', 'geometry_wkt']])
con.register("sensors", gdf_sensors[['id', 'value', 'geometry_wkt']])

# Create an output directory
os.makedirs("data/output_maps", exist_ok=True)

# Mapping function
def plot_sensors_and_cities(sensors_df, cities_df, filename="map.html"):
    m = folium.Map(location=[45.46, 9.19], zoom_start=10)
    for _, row in cities_df.iterrows():
        geom = wkt.loads(row['geometry_wkt'])
        folium.GeoJson(geom, name=row['name'], tooltip=row['name']).add_to(m)
    for _, row in sensors_df.iterrows():
        geom = wkt.loads(row['geometry_wkt'])
        folium.CircleMarker(
            location=[geom.y, geom.x],
            radius=5,
            popup=f"Sensor ID: {row['id']}, Value: {row['value']}",
            color='red', fill=True
        ).add_to(m)
    m.save(f"data/output_maps/{filename}")
    print(f"Map saved to data/output_maps/{filename}")

### Example 1: Sensors inside cities

This query selects all sensors that are geographically located inside city polygons

In [18]:
# Query: Find sensors inside cities using ST_Within
result = con.execute("""
    SELECT s.id, s.value, s.geometry_wkt, c.name AS city, c.geometry_wkt AS city_geometry
    FROM sensors s, cities c
    WHERE ST_Within(ST_GeomFromText(s.geometry_wkt), ST_GeomFromText(c.geometry_wkt))
""").fetchdf()

# Show result and generate map
display(result)
cities_map_df = result[['city', 'city_geometry']].drop_duplicates().rename(columns={'city': 'name', 'city_geometry': 'geometry_wkt'})
plot_sensors_and_cities(result[['id', 'value', 'geometry_wkt']], cities_map_df, filename="sensors_in_cities.html")

,id,value,geometry_wkt,city,city_geometry
0,1,34.2,POINT (9.1 45.1),Milano,"POLYGON ((9 45, 9.2 45, 9.2 45.2, 9 45.2, 9 45))"
1,3,40.5,POINT (9.05 45.15),Milano,"POLYGON ((9 45, 9.2 45, 9.2 45.2, 9 45.2, 9 45))"


Map saved to data/output_maps/sensors_in_cities.html


### Example 2: Sensors within 1km of Milan center

Uses ST_Distance() to select all sensors near a fixed point.

In [24]:
milano_center = "POINT(9.19 45.46)"  # EPSG:4326

# Query: Find sensors within 1 km of the city's center
result = con.execute("""
    SELECT id, value, geometry_wkt
    FROM sensors
    WHERE ST_Distance(ST_GeomFromText(geometry_wkt), ST_GeomFromText(?)) <= 1000
""", [milano_center]).fetchdf()

# Show result and generate map
display(result)
plot_sensors_and_cities(result, cities.iloc[[0]], filename="sensors_within_1km_milan.html")

,id,value,geometry_wkt
0,1,34.2,POINT (9.1 45.1)
1,2,22.8,POINT (9.5 45.5)
2,3,40.5,POINT (9.05 45.15)


Map saved to data/output_maps/sensors_within_1km_milan.html


### Example 3: Distance to city centroid

Calculates the distance from each sensor to the centroid of the first city (e.g. Milan).

In [25]:
# Get the WKT geometry of the first city
city_geom = cities['geometry_wkt'].iloc[0]

# Compute distance from each sensor to the city's centroid
result = con.execute("""
    SELECT id, value,
           ST_Distance(ST_GeomFromText(geometry_wkt), ST_Centroid(ST_GeomFromText(?))) AS distance_to_center
    FROM sensors
""", [city_geom]).fetchdf()

# Show the result as a table
display(result)

,id,value,distance_to_center
0,1,34.2,0.000000
1,2,22.8,0.565685
2,3,40.5,0.070711


### Example 4: Sensors outside any city

Finds sensors that are not contained in any city using a LEFT JOIN + IS NULL.

In [26]:
# Query sensors that are NOT inside any city polygon (left join with null)
result = con.execute("""
    SELECT s.id, s.value, s.geometry_wkt
    FROM sensors s
    LEFT JOIN cities c
    ON ST_Within(ST_GeomFromText(s.geometry_wkt), ST_GeomFromText(c.geometry_wkt))
    WHERE c.name IS NULL
""").fetchdf()

# Show the result
display(result)

# Visualize sensors outside cities on map
plot_sensors_and_cities(result, cities, filename="sensors_outside_cities.html")

,id,value,geometry_wkt
0,2,22.8,POINT (9.5 45.5)


Map saved to data/output_maps/sensors_outside_cities.html


### Example 5: Sensor buffers intersecting cities

Creates 5km circular buffers around sensors and checks which ones intersect city polygons.

In [27]:
# Query sensors whose 5 km buffer intersects any city polygon
result = con.execute("""
    SELECT s.id, s.value, s.geometry_wkt, c.name AS city, c.geometry_wkt AS city_geometry
    FROM sensors s, cities c
    WHERE ST_Intersects(
        ST_Buffer(ST_GeomFromText(s.geometry_wkt), 5000),
        ST_GeomFromText(c.geometry_wkt)
    )
""").fetchdf()

# Show the result
display(result)

# Prepare city geometries for map (remove duplicates)
buffer_map_df = result[['city', 'city_geometry']].drop_duplicates().rename(columns={
    'city': 'name', 'city_geometry': 'geometry_wkt'
})

# Visualize buffers intersecting cities
plot_sensors_and_cities(result[['id', 'value', 'geometry_wkt']], buffer_map_df,
                        filename="sensor_buffers_intersecting_cities.html")

,id,value,geometry_wkt,city,city_geometry
0,1,34.2,POINT (9.1 45.1),Milano,"POLYGON ((9 45, 9.2 45, 9.2 45.2, 9 45.2, 9 45))"
1,2,22.8,POINT (9.5 45.5),Milano,"POLYGON ((9 45, 9.2 45, 9.2 45.2, 9 45.2, 9 45))"
2,3,40.5,POINT (9.05 45.15),Milano,"POLYGON ((9 45, 9.2 45, 9.2 45.2, 9 45.2, 9 45))"


Map saved to data/output_maps/sensor_buffers_intersecting_cities.html
